# Creating a new planner in nuPlan <a name="introduction"></a>


## Setup

In [1]:
import asyncio
import itertools
import logging
import random
import nest_asyncio

nest_asyncio.apply()  # must be run at the start of every notebook to patch the event loop to allow nesting, eg. so we can run asyncio.run from within a notebook.

# from IPython.core.display import display, HTML
# from bokeh.io import output_notebook

In [2]:
import os
from pathlib import Path
import tempfile

from shutil import rmtree
from typing import List, Optional, Union

In [3]:
# nest_asyncio.apply()
# output_notebook()
# display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# %cd ..

# Adjust your env variables here if nescesary

%env NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=../../data/nuplan/maps
%env NUPLAN_EXP_ROOT=../../data/nuplan/exp
%env NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/
%pwd

env: NUPLAN_DATA_ROOT=../../../datasets/nuplan/dataset
env: NUPLAN_MAPS_ROOT=../../data/nuplan/maps
env: NUPLAN_EXP_ROOT=../../data/nuplan/exp
env: NUPLAN_DEVKIT_ROOT=../../nuplan-devkit/


'/home/ehdykhne/nuplan-devkit/experiments'

In [4]:
import hydra
import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf

from nuplan.common.utils.s3_utils import is_s3_path
from nuplan.planning.script.builders.simulation_builder import build_simulations
from nuplan.planning.script.builders.simulation_callback_builder import (
    build_callbacks_worker,
    build_simulation_callbacks,
)
from nuplan.planning.script.utils import (
    run_runners,
    set_default_path,
    set_up_common_builder,
)
from nuplan.planning.simulation.planner.abstract_planner import AbstractPlanner

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# If set, use the env. variable to overwrite the default dataset and experiment paths
set_default_path()

# If set, use the env. variable to overwrite the Hydra config
CONFIG_PATH = os.getenv("NUPLAN_HYDRA_CONFIG_PATH", "config/simulation")

if os.environ.get("NUPLAN_HYDRA_CONFIG_PATH") is not None:
    CONFIG_PATH = os.path.join("../../../../", CONFIG_PATH)

if os.path.basename(CONFIG_PATH) != "simulation":
    CONFIG_PATH = os.path.join(CONFIG_PATH, "simulation")
CONFIG_NAME = "default_simulation"

# Simulating the planner <a name="simulation"></a>

## Prepare the simulation config

In [5]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(
    os.getenv("NUPLAN_TUTORIAL_PATH", ""), "../nuplan/planning/script"
)
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

# Create a temporary directory to store the simulation artifacts

scenario_types = [
    "accelerating_at_traffic_light_with_lead",
    "crossed_by_bike",
    "crossed_by_vehicle",
    # "on_intersection",
    # "on_stopline_crosswalk",
    # "on_stopline_stop_sign",
    # "on_stopline_traffic_light",
    "on_traffic_light_intersection",
    # "starting_protected_cross_turn",
    # "starting_protected_noncross_turn",
    # "starting_right_turn",
    "starting_straight_stop_sign_intersection_traversal",
    "starting_straight_traffic_light_intersection_traversal",
    # "starting_u_turn",
    "starting_unprotected_cross_turn",
    "starting_unprotected_noncross_turn",
    # "stationary_at_crosswalk",
    # "stationary_at_traffic_light_with_lead",
    # "stationary_at_traffic_light_without_lead",
    # "traversing_crosswalk",
    # "traversing_intersection",
    "traversing_traffic_light_intersection",
]

# scenario_types = ['stationary_at_traffic_light_without_lead']

scenario_builder = "val"  # [nuplan (uses trainval), nuplan_mini, test, val, train_boston, train_pittsburgh, train_singapore]
DATASET_PARAMS = [
    f"scenario_builder={scenario_builder}",
    "scenario_filter=all_scenarios",  # [all_scenarios, val14_split]
    # f"scenario_filter.scenario_types={scenario_types}",  # there are 70 scenario types in the trainingset and 58 in the validation set including "unknown" which make up the majority
    # "scenario_filter.ego_displacement_minimum_m=10",  # use scenarios where the ego vehicle moves at least 10m
    #    'scenario_filter.remove_invalid_goals=true',  # remove scenarios where the goal is not invalid
    # "scenario_filter.ego_start_speed_threshold=5",  # Exclusive threshold that the ego's speed must rise above (meters per second) for scenario to be kept
    #    'scenario_filter.stop_speed_threshold=10',  # Inclusive threshold that the ego's speed must fall below (meters per second) for scenario to be kept:
    # "scenario_filter.map_names=[us-pa-pittsburgh-hazelwood]",  # [sg-one-north, us-ma-boston, us-pa-pittsburgh-hazelwood, us-nv-las-vegas-strip]
    # "scenario_filter.num_scenarios_per_type=200",  # use 10 scenarios per scenario type
    "scenario_filter.scenario_tokens=['df4441167a6f54f8']",  # List of scenarios to include (token)
    #    'scenario_filter.log_names=["2021.08.24.18.07.48_veh-45_01504_01722"]', # specific scenrios to simulate
    # "scenario_filter.limit_total_scenarios=0.05",  # use n total scenarios if int, or if float smaller than 1, use n as a fraction of total scenarios (changes sampling frequency, unchanged leaves the frequency at 20Hz)
]
ckpt_dir = "/home/ehdykhne/nuplan-devkit/experiments/pretrained_checkpoints/gc_pgp_checkpoint.ckpt"
hybrid_ckpt = "/home/sacardoz/checkpoints/pdm_offset_checkpoint.ckpt"
#'/home/sacardoz/checkpoints/urbandriver_checkpoint.ckpt'
# "/home/sacardoz/tutorial_vector_framework/training_simple_vector_experiment/train_default_simple_vector/2023.11.23.09.55.21/best_model/epoch.ckpt"
# "/home/sacardoz/training_raster_experiment/train_default_raster/2023.11.23.07.36.36/best_model/epoch.ckpt"
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
cfg = hydra.compose(
    config_name=simulation_hydra_paths.config_name,
    overrides=[
        "+simulation=closed_loop_multiagent",  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents, closed_loop_multiagent]
        "planner=ml_planner",
        # f"planner.pdm_closed_planner.checkpoint_path={hybrid_ckpt}",
        #'observation.model_config=${model}',
        # f'observation.checkpoint_path={ckpt_dir}',
        "model=gc_pgp_model",
        "planner.ml_planner.model_config=${model}",
        f"planner.ml_planner.checkpoint_path={ckpt_dir}",
        f"observation.planner_type=ml",
        "observation.model_config=${model}",
        f"observation.checkpoint_path={ckpt_dir}",
        # f"observation.pdm_hybrid_ckpt={hybrid_ckpt}",
        f"observation.occlusion_cfg.occlusion=true",
        f"observation.occlusion_cfg.manager_type=wedge",
        f"+observation.occlusion_cfg.uncloak_reaction_time=1.5",
        f"+observation.occlusion_cfg.notice_threshold=1.0",
        "+occlusion=true",
        "+occlusion.manager_type=wedge",  # options: [range, shadow, wedge]
        "+occlusion.uncloak_reaction_time=1.5",
        "+occlusion.notice_threshold=1.0",
        "worker=sequential",  # [sequential, ray_distributed]
        "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
        *DATASET_PARAMS,
    ],
)

output_folder = cfg.output_dir
print('output_folder = "' + output_folder + '"')

# '+simulation=closed_loop_multiagent',
#     #'model=urban_driver_open_loop_model',
#     'planner=pdm_hybrid_planner',
#     f"planner.pdm_hybrid_planner.checkpoint_path={hybrid_ckpt}" ,
#     #'observation.model_config=${model}',
#     #f'observation.checkpoint_path={ckpt_dir}',
#     f'observation.planner_type=pdm_hybrid',
#     f'observation.pdm_hybrid_ckpt={hybrid_ckpt}',
#     f'observation.occlusion_cfg.occlusion=true',
#     f'observation.occlusion_cfg.manager_type=wedge',
#     'worker=sequential',
#     '+occlusion=true',
#     '+occlusion.manager_type=wedge', #options: [range, shadow, wedge]
#     "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",

output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.01.03.14.19.08"


In [7]:
from nuplan.planning.script.run_simulation import build_simulation_runners
from nuplan.common.actor_state.tracked_objects_types import (
    AGENT_TYPES,
    STATIC_OBJECT_TYPES,
    TrackedObjectType,
)

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
runners, common_builder, cfg = build_simulation_runners(cfg)

Global seed set to 0
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback...
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback: 4...DONE!


2024-01-02 11:51:21,204 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2024-01-02 11:51:21,205 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: Sequential
2024-01-02 11:51:21,205 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 1
Number of GPUs per node: 0
Number of threads across all nodes: 1
2024-01-02 11:51:21,205 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2024-01-02 11:51:21,205 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2024-01-02 11:51:21,205 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are stored: ../../data/nuplan/exp/exp/simulation/closed_loop

In [8]:
runner = runners[0]

In [9]:
from nuplan.common.actor_state.agent import Agent
from nuplan.common.actor_state.oriented_box import OrientedBox
from nuplan.common.actor_state.scene_object import SceneObjectMetadata
from nuplan.common.actor_state.state_representation import StateSE2, StateVector2D
import math

speed = 5.2
angle = 1.25
inserted_agent = Agent(
    tracked_object_type=TrackedObjectType.VEHICLE,
    oriented_box=OrientedBox(StateSE2(587505.03, 4475691.75, angle), 5, 2, 2),
    velocity=StateVector2D(speed * math.cos(angle), speed * math.sin(angle)),
    metadata=SceneObjectMetadata(1623707858950113, "inserted", -2, "inserted"),
    angular_velocity=0.0,
)

inserted_goal = StateSE2(587567, 4475882, 1.25)
insertion_time = 0.0

In [10]:
runner.simulation.callback.on_simulation_start(runner.simulation.setup)

# Initialize all simulations
runner._initialize()

while runner.simulation.is_simulation_running():
    if runner.simulation._time_controller.get_iteration().index == insertion_time:
        iter = runner.simulation._time_controller.get_iteration()
        runner.simulation._observations.add_agent_to_scene(
            inserted_agent, inserted_goal, iter.time_point, runner.simulation
        )

    # Execute specific callback
    runner.simulation.callback.on_step_start(runner.simulation.setup, runner.planner)

    # Perform step
    planner_input = runner._simulation.get_planner_input()

    # Execute specific call+back
    runner._simulation.callback.on_planner_start(
        runner.simulation.setup, runner.planner
    )

    # Plan path based on all planner's inputs
    trajectory = runner.planner.compute_trajectory(planner_input)

    # Propagate simulation based on planner trajectory
    runner._simulation.callback.on_planner_end(
        runner.simulation.setup, runner.planner, trajectory
    )

    iteration = runner.simulation._time_controller.get_iteration()
    print(iteration)
    runner.simulation.propagate(trajectory)
    # Execute specific callback
    runner.simulation.callback.on_step_end(
        runner.simulation.setup, runner.planner, runner.simulation.history.last()
    )

runner.simulation.callback.on_simulation_end(
    runner.simulation.setup, runner.planner, runner.simulation.history
)

hi 0.0
SimulationIteration(time_point=TimePoint(time_us=1629836927500525), index=0)
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.0
hi 0.09994697570800781
SimulationIteration(time_point=TimePoint(time_us=1629836927600472), index=1)
hi 0.09994697570800781
hi 0.09994697570800781
hi 0.09994697570800781
hi 0.09994697570800781
hi 0.09994697570800781
hi 0.09994697570800781
hi 0.09994697570800781
hi 0.09994697570800781
hi 0.19988703727722168
SimulationIteration(time_point=TimePoint(time_us=1629836927700412), index=2)
hi 0.19988703727722168
hi 0.19988703727722168
hi 0.19988703727722168
hi 0.19988703727722168
hi 0.19988703727722168
hi 0.19988703727722168
hi 0.19988703727722168
hi 0.19988703727722168
hi 0.2998189926147461
SimulationIteration(time_point=TimePoint(time_us=1629836927800344), index=3)
hi 0.2998189926147461
hi 0.2998189926147461
hi 0.29981899261

In [6]:
scenario_builder = "val"  # [nuplan (uses trainval), nuplan_mini, test, val, train_boston, train_pittsburgh, train_singapore]
# output_folder = "../../data/nuplan/exp/exp/simulation/open_loop_boxes/2023.12.31.02.58.31"

# output_folder = (
#     "../../data/nuplan/exp/exp/simulation/open_loop_boxes/2023.12.31.05.00.39"
# )

output_folder = (
    "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.01.02.11.51.20"
)


# without injection
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.01.02.00.44.56"

# with injection
# output_folder = (
#     "../../data/nuplan/exp/exp/simulation/closed_loop_multiagent/2024.01.02.02.37.00"
# )

# output_folder = (
#     "../../data/nuplan/exp/exp/simulation/open_loop_boxes/2023.12.20.13.59.34"
# )
# output_folder = '../../data/nuplan/exp/exp/simulation/open_loop_boxes/2023.12.09.21.19.06'
# output_folder = "../../data/nuplan/exp/exp/simulation/closed_loop_reactive_agents/2023.12.09.21.59.48"
# Location of path with all nuBoard configs
CONFIG_PATH = "../nuplan/planning/script/config/nuboard"
CONFIG_NAME = "default_nuboard"

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(
    config_name=CONFIG_NAME,
    overrides=[
        f"scenario_builder={scenario_builder}",  # set the database (same as simulation) used to fetch data for visualization
        f"simulation_path={output_folder}",  # [output_folder, output_folder_alt] nuboard file path(s), if left empty the user can open the file inside nuBoard
    ],
)

In [7]:
from nuplan.planning.script.run_nuboard import main as main_nuboard

# Run nuBoard
main_nuboard(cfg)

INFO:nuplan.planning.script.builders.scenario_building_builder:Building AbstractScenarioBuilder...
INFO:nuplan.planning.script.builders.scenario_building_builder:Building AbstractScenarioBuilder...DONE!
INFO:nuplan.planning.nuboard.nuboard:Opening Bokeh application on http://localhost:5006/
INFO:nuplan.planning.nuboard.nuboard:Async rendering is set to: True
INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.3.3)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


INFO:nuplan.planning.nuboard.base.experiment_file_data:Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
INFO:nuplan.planning.nuboard.base.simulation_tile:Minimum frame time=0.017 s
INFO:nuplan.planning.nuboard.tabs.scenario_tab:Rending scenario plot takes 0.0005 seconds.
INFO:tornado.access:200 GET / (127.0.0.1) 383.31ms
INFO:tornado.access:200 GET / (127.0.0.1) 383.31ms
INFO:tornado.access:101 GET /ws (127.0.0.1) 0.50ms
INFO:tornado.access:101 GET /ws (127.0.0.1) 0.50ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created
INFO:nuplan.planning.nuboard.base.experiment_file_data:Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.
INFO:nuplan.planning.nuboard.base.simulation_tile:Minimum frame time=0.017 s
INFO:nuplan.planning.nuboard.tabs.sc

In [ ]:
from tutorials.utils.tutorial_utils import visualize_history

from IPython.core.display import display, HTML
from bokeh.io import output_notebook

/tmp/ipykernel_2434612/3982088180.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
nest_asyncio.apply()
output_notebook()
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

Loading BokehJS ...

In [ ]:
visualize_history(runner.simulation._history, runner.scenario, bokeh_port=5006)

NameError: name 'visualize_history' is not defined